In [2]:
import pandas as pd
import numpy as np

from IPython.display import Markdown, display
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import cluster, datasets
from sklearn.cluster import KMeans
import random
import pickle


In [3]:
#Importing required file


In [4]:
films = pd.read_csv('films_w_awards_cluster.csv', index_col =False)


/var/folders/0r/l856hq9x61zf5p8g8vxjs_dh0000gn/T/ipykernel_768/1553699301.py:1: DtypeWarning: Columns (25,26,27,28,29,30,31,32,33,34,35,36,37,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  films = pd.read_csv('films_w_awards_cluster.csv', index_col =False)


In [5]:
#Importing scaler and kmeans models


In [6]:
kmeans = pickle.load(open('kmeans_model.pkl','rb'))


In [7]:
scaler = pickle.load(open('scaler_model.pkl','rb'))


In [8]:
encoder = pickle.load(open('encoder_model.pkl','rb'))


In [92]:
# Ask for number of movie names to enter
num_movies = int(input("How many people are you? "))

# Create an empty list to store movie names
movie_names = []

# Loop to get movie names based on the number entered
for i in range(num_movies):
    movie_name = input("Please, enter movie you love (original name) #{}: ".format(i+1))
    movie_names.append(movie_name.lower())

# Search for the movie names in the 'films' dataframe
for user_input in movie_names:
    search_result = films[films['movie_name'].str.lower().str.contains(user_input, case=False, na=False)]

    if len(search_result) > 0:
        # If multiple options found, ask user to choose one
        if len(search_result) > 1:
            print("We have found different movies for that name! Please choose one from the list:")
            for i, movie in enumerate(search_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].iterrows(), start=1):
                print("{}. {}".format(i, movie[1]['movie_name']))
            chosen_movie_index = int(input("Enter the number corresponding to the movie you want to select: "))
            chosen_movie_result = search_result.iloc[chosen_movie_index-1]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("You have chosen: {}".format(chosen_movie_name))
        else:
            chosen_movie_result = search_result.iloc[0]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("You have chosen: {}".format(chosen_movie_name))

        # Get the cluster of the chosen movie
        cluster = chosen_movie_result['cluster']
        # Get the year_film of the chosen movie
        year_film = chosen_movie_result['year_film']

        # Convert year_film to integer
        year_film = int(year_film)

        print("Searched movie:")
        print(chosen_movie_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']])

        # Filter films dataframe by cluster
        films_in_same_cluster = films[films['cluster'] == cluster]

        if len(films_in_same_cluster) > 0:
            # Filter films in same cluster with close year_film
            films_close_year = films_in_same_cluster[(films_in_same_cluster['year_film'] >= year_film - 5) & (films_in_same_cluster['year_film'] <= year_film + 5)]
            if len(films_close_year) > 0:
                # Remove chosen movie from the recommendations
                films_close_year = films_close_year[films_close_year['movie_name'] != chosen_movie_name]
                # Print a random film from the same cluster with close year_film
                random_film = films_close_year.sample(n=1)
            else:
                # Print a random film from the same cluster
                random_film = films_in_same_cluster.sample(n=1)
        else:
            # Print a random film from the entire dataframe
            random_film = films.sample(n=1)

        # Extract required columns
        output = random_film[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].copy()
        # Convert movie_id to string and concatenate with IMDb URL
        output['imdb_url'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

        print("Recommended movie:")
        print(output)
        
    else:
        print("No movie found for the given input: {}".format(user_input))


How many people are you? 2
Please, enter movie you love (original name) #1: jurassic park
Please, enter movie you love (original name) #2: the mask
We have found different movies for that name! Please choose one from the list:
1. Jurassic Park
2. The Lost World: Jurassic Park
3. Jurassic Park III
4. Rifftrax: Jurassic Park
5. Jurassic Park: Operation Rebirth
Enter the number corresponding to the movie you want to select: 1
You have chosen: Jurassic Park
Searched movie:
movie_name       Jurassic Park
year_film                 1993
director      Steven Spielberg
runtime                    127
rating                     8.2
votes                   998365
movie_id             tt0107290
Name: 70002, dtype: object
Recommended movie:
                movie_name  year_film          director  runtime  rating  \
77290  Saving Private Ryan       1998  Steven Spielberg      169     8.6   

         votes   movie_id                              imdb_url  
77290  1402550  tt0120815  https://www.imdb.

In [72]:
# Ask for number of movie names to enter
num_movies = int(input("How many movies do you love? "))

# Create an empty list to store movie names
movie_names = []

# Loop to get movie names based on the number entered
for i in range(num_movies):
    movie_name = input("Please enter the name of movie #{}: ".format(i+1))
    movie_names.append(movie_name.lower())

# Search for the movie names in the 'films' dataframe
for user_input in movie_names:
    search_result = films[films['movie_name'].str.lower().str.contains(user_input, case=False, na=False)]

    if len(search_result) > 0:
        # If multiple options found, ask user to choose one
        if len(search_result) > 1:
            print("We have found different movies with that name! Please choose one from the list:")
            for i, movie in enumerate(search_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].iterrows(), start=1):
                print("{}. {}".format(i, movie[1]['movie_name']))
            chosen_movie_index = int(input("Enter the number corresponding to the movie you want to select: "))
            chosen_movie_result = search_result.iloc[chosen_movie_index-1]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("You have chosen: {}".format(chosen_movie_name))
        else:
            chosen_movie_result = search_result.iloc[0]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("You have chosen: {}".format(chosen_movie_name))

        # Get the cluster of the chosen movie
        cluster = chosen_movie_result['cluster']
        # Get the year_film of the chosen movie
        year_film = chosen_movie_result['year_film']

        # Convert year_film to integer
        year_film = int(year_film)

        print("Searched movie:")
        print(chosen_movie_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']])

        # Filter films dataframe by cluster
        films_in_same_cluster = films[films['cluster'] == cluster]

        if len(films_in_same_cluster) > 0:
            # Filter films in same cluster with close year_film
            films_close_year = films_in_same_cluster[(films_in_same_cluster['year_film'] >= year_film - 5) & (films_in_same_cluster['year_film'] <= year_film + 5)]
            if len(films_close_year) > 0:
                # Remove chosen movie from the recommendations
                films_close_year = films_close_year[films_close_year['movie_name'] != chosen_movie_name]
                # Print a random film from the same cluster with close year_film
                random_film = films_close_year.sample(n=1)
            else:
                # Print a random film from the same cluster
                random_film = films_in_same_cluster.sample(n=1)
        else:
            # Print a random film from the entire dataframe
            random_film = films.sample(n=1)

        # Extract required columns
        output = random_film[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].copy()
        # Convert movie_id to string and concatenate with IMDb URL
        output['imdb_url'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

        print("Recommended movie:")
        print(output)
        
    else:
        print("No movie found for the given input: {}".format(user_input))


How many movies do you love? 2
Please enter the name of movie #1: jurassic world
Please enter the name of movie #2: godfather
We have found different movies with that name! Please choose one from the list:
1. Jurassic World
2. Rifftrax: Jurassic World
3. Jurassic World: Fallen Kingdom
4. Rifftrax: Jurassic World: Fallen Kingdom
5. Jurassic World: Dominion
Enter the number corresponding to the movie you want to select: 1
You have chosen: Jurassic World
Searched movie:
movie_name     Jurassic World
year_film                2015
director      Colin Trevorrow
runtime                   124
rating                    6.9
votes                  649333
movie_id            tt0369610
Name: 128091, dtype: object
Recommended movie:
           movie_name  year_film   director  runtime  rating   votes  \
116560  The Conjuring       2013  James Wan      112     7.5  516522   

         movie_id                              imdb_url  
116560  tt1457767  https://www.imdb.com/title/tt1457767  
We have fo

ValueError: a must be greater than 0 unless no samples are taken

In [82]:
import pandas as pd

# Ask for number of movie names to enter
num_movies = int(input("How many movies do you love? "))

# Create an empty list to store movie names
movie_names = []

# Loop to get movie names based on the number entered
for i in range(num_movies):
    movie_name = input("Please enter the name of movie #{}: ".format(i+1))
    movie_names.append(movie_name.lower())

# Search for the movie names in the 'films' dataframe
for user_input in movie_names:
    search_result = films[films['movie_name'].str.lower().str.contains(user_input, case=False, na=False)]

    if len(search_result) > 0:
        # If multiple options found, ask user to choose one
        if len(search_result) > 1:
            print("We have found different movies with that name! Please choose one from the list:")
            for i, movie in enumerate(search_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].iterrows(), start=1):
                print("{}. {}".format(i, movie[1]['movie_name']))
            chosen_movie_index = int(input("Enter the number corresponding to the movie you want to select: "))
            chosen_movie_result = search_result.iloc[chosen_movie_index-1]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("You have chosen: {}".format(chosen_movie_name))
        else:
            chosen_movie_result = search_result.iloc[0]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("You have chosen: {}".format(chosen_movie_name))

        # Get the cluster of the chosen movie
        cluster = chosen_movie_result['cluster']
        # Get the year_film of the chosen movie
        year_film = chosen_movie_result['year_film']

        # Convert year_film to integer
        year_film = int(year_film)

        print("Searched movie:")
        print(chosen_movie_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']])

        # Filter films dataframe by cluster
        films_in_same_cluster = films[films['cluster'] == cluster]

        if len(films_in_same_cluster) > 0:
            # Filter films in same cluster with close year_film
            films_close_year = films_in_same_cluster[(films_in_same_cluster['year_film'] >= year_film - 5) & (films_in_same_cluster['year_film'] <= year_film + 5)]
            if len(films_close_year) > 0:
                # Remove chosen movie from the recommendations
                films_close_year = films_close_year[films_close_year['movie_name'] != chosen_movie_name]
                # Print a random film from the same cluster with close year_film
                random_film = films_close_year.sample(n=1)
            else:
                # Print a random film from the same cluster
                random_film = films_in_same_cluster.sample(n=1)
        else:
            # Print a random film from the entire dataframe
            random_film = films.sample(n=1)

        # Extract required columns
        output = random_film[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].copy()
        # Convert movie_id to string and concatenate with IMDb URL
        output['imdb_url'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

        print("Recommended movie:")
        print(output)
        
    else:
        print("No movie found for the given input: {}".format(user_input))

# Ask the user if they want to search for more movies
while True:
    another_search = input("Do you want to search for more movies? (Yes/No): ")
    if another_search.lower() == 'yes':
        # Ask for number of movie names to enter
        num_movies = int(input("How many movies do you love? "))

        # Create an empty list to store movie names
        movie_names = []

        # Loop to get movie names based on the number entered
        for i in range(num_movies):
            movie_name = input("Please enter the name of movie #{}: ".format(i+1))
            movie_names.append(movie_name.lower())

        # Search for the movie names in the 'films' dataframe
        for user_input in movie_names:
            search_result = films[films['movie_name'].str.lower().str.contains(user_input, case=False, na=False)]

            if len(search_result) > 0:
                # If multiple options found, ask user to choose one
                if len(search_result) > 1:
                    print("We have found different movies with that name! Please choose one from the list:")
                    for i, movie in enumerate(search_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].iterrows(), start=1):
                        print("{}. {}".format(i, movie[1]['movie_name']))
                    chosen_movie_index = int(input("Enter the number corresponding to the movie you want to select: "))
                    chosen_movie_result = search_result.iloc[chosen_movie_index-1]
                    chosen_movie_name = chosen_movie_result['movie_name']
                    print("You have chosen: {}".format(chosen_movie_name))
                else:
                    chosen_movie_result = search_result.iloc[0]
                    chosen_movie_name = chosen_movie_result['movie_name']
                    print("You have chosen: {}".format(chosen_movie_name))

                # Get the cluster of the chosen movie
                cluster = chosen_movie_result['cluster']
                # Get the year_film of the chosen movie
                year_film = chosen_movie_result['year_film']

                # Convert year_film to integer
                year_film = int(year_film)

                print("Searched movie:")
                print(chosen_movie_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']])

                # Filter films dataframe by cluster
                films_in_same_cluster = films[films['cluster'] == cluster]

                if len(films_in_same_cluster) > 0:
                    # Filter films in same cluster with close year_film
                    films_close_year = films_in_same_cluster[(films_in_same_cluster['year_film'] >= year_film - 5) & (films_in_same_cluster['year_film'] <= year_film + 5)]
                    if len(films_close_year) > 0:
                        # Remove chosen movie from the recommendations
                        films_close_year = films_close_year[films_close_year['movie_name'] != chosen_movie_name]
                        # Print a random film from the same cluster with close year_film
                        random_film = films_close_year.sample(n=1)
                    else:
                        # Print a random film from the same cluster
                        random_film =films_in_same_cluster.sample(n=1)
                else:
                    # If no films found in the same cluster, print a random film from the entire dataset
                    random_film = films.sample(n=1)

                # Extract relevant information from the random film
                output = random_film[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']]
                
                # Convert movie_id to string and concatenate with IMDb URL
                output['imdb_url'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

                print("Recommended movie:")
                print(output)

            else:
                print("No movie found for the given input: {}".format(user_input))

    else:
        print("Thank you for using the movie recommendation system!")
        break

How many movies do you love? 2
Please enter the name of movie #1: jurassic park
Please enter the name of movie #2: godfather
We have found different movies with that name! Please choose one from the list:
1. Jurassic Park
2. The Lost World: Jurassic Park
3. Jurassic Park III
4. Rifftrax: Jurassic Park
5. Jurassic Park: Operation Rebirth
Enter the number corresponding to the movie you want to select: 1
You have chosen: Jurassic Park
Searched movie:
movie_name       Jurassic Park
year_film                 1993
director      Steven Spielberg
runtime                    127
rating                     8.2
votes                   998365
movie_id             tt0107290
Name: 70002, dtype: object
Recommended movie:
                       movie_name  year_film       director  runtime  rating  \
66050  Terminator 2: Judgment Day       1991  James Cameron      137     8.6   

         votes   movie_id                              imdb_url  
66050  1107922  tt0103064  https://www.imdb.com/title/tt01

ValueError: a must be greater than 0 unless no samples are taken

In [79]:
films.columns

Index(['year_film', 'movie_name', 'runtime', 'rating', 'votes',
       'osc_nominated_num', 'osc_won_num', 'cannes_won_num', 'director',
       'actor1', 'actor2', 'actor3', 'actor4', 'genre1', 'genre2', 'genre3',
       'osc_nominated_category1', 'osc_nominated_category2',
       'osc_nominated_category3', 'osc_nominated_category4',
       'osc_nominated_category5', 'osc_nominated_category6',
       'osc_nominated_category7', 'osc_nominated_category8',
       'osc_nominated_category9', 'osc_nominated_category10',
       'osc_nominated_category11', 'osc_nominated_category12',
       'osc_nominated_category13', 'osc_nominated_category14',
       'osc_nominated_category15', 'osc_nominated_category16',
       'osc_nominated_category17', 'osc_nominated_category18',
       'osc_nominated_category19', 'osc_nominated_category20',
       'osc_nominated_category21', 'osc_nominated_category22',
       'osc_won_category1', 'osc_won_category2', 'osc_won_category3',
       'osc_won_category4', 'osc

In [90]:
import pandas as pd

# Pregunta al usuario cuántas personas hay en el grupo
num_people = int(input("How many people are you? (Enter a number between 1 and 5): "))

# Validación del número de personas en el grupo
while num_people < 1 or num_people > 5:
    print("Invalid input. Please enter a number between 1 and 5.")
    num_people = int(input("How many people are you? (Enter a number between 1 and 5): "))

# Crear una lista para almacenar las películas elegidas por cada persona del grupo
movies_chosen = []

# Preguntar al usuario por las películas que le gusta a cada persona del grupo
for i in range(num_people):
    movie = input(f"Please enter a movie you love (original name) for person {i+1}: ")
    movies_chosen.append(movie.lower())  # Convertir el nombre de la película a minúsculas

# Filtrar las películas elegidas por el grupo en el dataframe 'films'
filtered_films = films[films['movie_name'].str.lower().isin(movies_chosen)]

# Validar si se encontraron todas las películas elegidas por el grupo
if len(filtered_films) < num_people:
    print("Some movies were not found. Restarting...")
else:
    # Obtener el 'cluster' más común de las películas elegidas por el grupo
    cluster_mode = filtered_films['cluster'].mode().values[0]

    # Si todos los clusters son diferentes, elegir el cluster de la película con el mayor valor en 'rating'
    if len(filtered_films['cluster'].unique()) == num_people:
        cluster_mode = filtered_films[filtered_films['rating'] == filtered_films['rating'].max()]['cluster'].values[0]

    # Obtener el año de la película más actual del grupo
    current_year = filtered_films['year_film'].max()

    # Filtrar las películas del mismo cluster y con el mismo año +/- 5 años
    recommended_films = filtered_films[(filtered_films['cluster'] == cluster_mode) & (filtered_films['year_film'].between(current_year-5, current_year+5))]

    # Si no se encontraron películas con el mismo cluster y año, elegir una película random del mismo año que la más actual
    if len(recommended_films) == 0:
        recommended_films = filtered_films[filtered_films['year_film'] == current_year].sample(n=1)

    # Filtrar las películas por director y actores
    director = filtered_films['director'].iloc[0]
    actors = filtered_films[['actor1', 'actor2', 'actor3', 'actor4']].values.flatten()
    recommended_films = recommended_films[(recommended_films['director'] == director) | (recommended_films['actor1'].isin(actors)) |
                                          (recommended_films['actor2'].isin(actors)) | (recommended_films['actor3'].isin(actors)) |
                                          (recommended_films['actor4'].isin(actors))]

#     # Ordenar las películas por rating, votes, osc_nominated_num y osc_won_num en orden descendente
#     output = output.sort_values(by=['rating', 'votes', 'osc_nominated_num', 'osc_won_num'], ascending=False)

    # Seleccionar las 3 primeras películas recomendadas
    output = output.head(3)

    # Crear el enlace de IMDb para cada película recomendada
    output['imdb_link'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

    # Seleccionar las columnas requeridas en el output
    output = output[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'description''description', 'imdb_link']]

    # Resetear el índice del dataframe
    output = output.reset_index(drop=True)

    # Mostrar el dataframe con las recomendaciones
    print("Movie Recommendations:")
    print(output)

How many people are you? (Enter a number between 1 and 5): 1
Please enter a movie you love (original name) for person 1: jurassic park


KeyError: 'movie_id'

In [1]:
import streamlit as st
import pandas as pd
import numpy as np


In [2]:
st.title('Uber pickups in NYC')

2023-04-18 12:13:38.920 
  command:

    streamlit run /Users/juan/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [3]:
streamlit run /Users/juan/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py

SyntaxError: invalid syntax (3787920738.py, line 1)

In [4]:
streamlit run /Users/juan/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]

SyntaxError: invalid syntax (998644944.py, line 1)

In [5]:
streamlit run uber_pickups.py


SyntaxError: invalid syntax (1460881132.py, line 1)